In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# import seaborn as sns
# sns.set()
# sns.set_style("white")
# sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
from pathlib import Path

import geopy
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic

In [14]:
df = pd.read_csv("../all_station_id.csv")

In [3]:
df2 = pd.read_csv("../boston_2016_distance_duration_speed.csv")

In [4]:
dingo = (df2.where(lambda x: (x.start_station_id == 8.0), axis = 0))
len(df2) - dingo['start_station_id'].isnull().sum()

4174

In [5]:
DasDingo = []

In [ ]:
for i in df.start_station_id:
    dingo = (df2.where(lambda x: (x.start_station_id == i), axis = 0))
    DasDingo.append(len(df2) - dingo['start_station_id'].isnull().sum())

In [7]:
print(DasDingo)

[78, 4690, 9005, 7090, 11713, 4589, 4174, 9549, 9668, 8595, 8022, 7793, 8788, 2622, 15091, 6582, 7246, 10970, 12111, 31524, 14589, 7738, 8452, 7972, 5020, 3728, 6317, 5405, 9267, 13918, 9473, 16721, 1325, 9078, 9242, 10873, 8672, 14032, 11103, 7810, 10154, 13260, 14041, 8065, 7697, 8599, 2842, 9263, 17144, 11712, 12033, 1494, 779, 14385, 7975, 15856, 12808, 3344, 9663, 3010, 5085, 35654, 24127, 6259, 8815, 5802, 10675, 11801, 23182, 11489, 14290, 2193, 5543, 6483, 21185, 10500, 2932, 9143, 5364, 10363, 13167, 7881, 12069, 14672, 1065, 1543, 5526, 11471, 10049, 9841, 11410, 3530, 10470, 2283, 3221, 6411, 12290, 513, 14424, 8604, 12833, 9486, 1710, 1372, 2455, 3234, 10232, 9713, 7159, 5538, 4030, 8833, 4866, 3611, 2461, 2657, 1055, 1047, 2484, 726, 3729, 2843, 5992, 3427, 4392, 1616, 935, 6161, 2908, 6640, 3145, 6305, 1888, 3040, 6279, 5170, 7602, 6514, 0, 0, 1745, 4685, 5050, 397, 4974, 1081, 2872, 440, 2417, 507, 1054, 1998, 3321, 11758, 15227, 17488, 2410, 408, 782, 565, 4711, 18004, 

In [18]:
#df['amout_of_trips_per_station'] = DasDingo(lambda x: (x.DasDingo), axis = 0)
df['amout_of_trips_per_station'] = pd.DataFrame (DasDingo, columns = ['amout_of_trips_per_station'])

In [20]:
df

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station
0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,78
1,26681,3,Colleges of the Fenway,42.340021,-71.100812,4690
2,97049,4,Tremont St. at Berkeley St.,42.345392,-71.069616,9005
3,161457,5,Northeastern U / North Parking Lot,42.341814,-71.090179,7090
4,103358,6,Cambridge St. at Joy St.,42.361174,-71.065142,11713
...,...,...,...,...,...,...
184,29748,215,The Eddy at New Street,42.370785,-71.044262,45
185,72713,216,Glendon St at Condor St,42.382447,-71.030232,43
186,71084,217,Orient Heights T Stop - Bennington St at Sarat...,42.386732,-71.006212,71
187,30227,218,Watermark Seaport - Boston Wharf Rd at Seaport...,42.351586,-71.045693,1414


In [21]:
filepath = Path('/Users/daniel/Desktop/boston_2016_distance_duration_speed.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath)

In [22]:
sum(df['amout_of_trips_per_station'])

1236192